In [1]:
import numpy as np

from ddnn.data import read_ML_cup, read_monks, train_valid_split, onehot_encoding
from ddnn.nn import *
from ddnn.utils import Dataset, Parameter

Test Data Modules

In [2]:
train = onehot_encoding(read_monks(1, "train"))

### Trainer Class

In [3]:
trainer = Estimator(
    NeuralNetwork(
        [
            LinearLayer((17, 4)),
            ActivationFunction(),
            LinearLayer((4, 1)),
            ActivationFunction("logistic"),
        ]
    ),
    batchsize=16,
    seed=123,
    optimizer=Optimizer(
        "SGD", learning_rate=0.1, l2_coefficient=1e-4, momentum_coefficient=0.8
    ),
)

In [7]:
trainer.train(train, n_epochs=200)

{'epoch': 101, 'loss': 0.04260817165738319}
{'epoch': 102, 'loss': 0.04381576950544911}
{'epoch': 103, 'loss': 0.04132383820690559}
{'epoch': 104, 'loss': 0.04132723484089104}
{'epoch': 105, 'loss': 0.04123517339796879}
{'epoch': 106, 'loss': 0.040201670223780205}
{'epoch': 107, 'loss': 0.04100820928146593}
{'epoch': 108, 'loss': 0.041717024578818596}
{'epoch': 109, 'loss': 0.04069669896433886}
{'epoch': 110, 'loss': 0.03892362864917116}
{'epoch': 111, 'loss': 0.03898772248102684}
{'epoch': 112, 'loss': 0.037713057099317375}
{'epoch': 113, 'loss': 0.0384050381387213}
{'epoch': 114, 'loss': 0.03830278086624776}
{'epoch': 115, 'loss': 0.03792452223527622}
{'epoch': 116, 'loss': 0.037142891375524706}
{'epoch': 117, 'loss': 0.038015218192513384}
{'epoch': 118, 'loss': 0.03854369537113984}
{'epoch': 119, 'loss': 0.03645183960634625}
{'epoch': 120, 'loss': 0.03552930709846025}
{'epoch': 121, 'loss': 0.03568224605842146}
{'epoch': 122, 'loss': 0.0352081183983849}
{'epoch': 123, 'loss': 0.0337

In [8]:
test = onehot_encoding(read_monks(1, "test"))
trainer.evaluate(["binary_accuracy"], test)

{'binary_accuracy': 1.0}